# CellTK Quickstarter

This notebook will walk through the general features of CellTK. Start by setting the correct path and importing CellTK.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

import celltk as ctk

The **Pipeline** is used to apply a series of **Operations** to the images captured by the microscope. For this example, point it to the path for some example images. *verbose* is True so that more feedback is shown to the user.

In [2]:
pipe = ctk.Pipeline(parent_folder='../examples/live_cell_example/', verbose=True)

2022-06-04 11:23:51,874 - Pipeline - INFO: Pipeline initiated.
2022-06-04 11:23:51,876 - Pipeline - INFO: Pipeline: <celltk.core.pipeline.Pipeline object at 0x14b3633b0>.
2022-06-04 11:23:51,877 - Pipeline - INFO: Parent folder: ../examples/live_cell_example/
2022-06-04 11:23:51,878 - Pipeline - INFO: Output folder: ../examples/live_cell_example/outputs
2022-06-04 11:23:51,879 - Pipeline - INFO: Image folder: ../examples/live_cell_example/
2022-06-04 11:23:51,879 - Pipeline - INFO: Mask folder: ../examples/live_cell_example/outputs
2022-06-04 11:23:51,880 - Pipeline - INFO: Array folder: ../examples/live_cell_example/outputs
2022-06-04 11:23:51,881 - Pipeline - INFO: Other inputs: [('name', 'live_cell_example'), ('frame_rng', None), ('skip_frames', None), ('overwrite', True), ('log_file', True), ('file_extension', 'tif')]


The information above displays where the **Pipeline** will look for images, as well as some general settings.

Next, define the image processing steps to complete on the images. In this example the image will first be processed by a convolutional neural net to predict the likelihood that a cell nucleus is present. This, and other preprocessing functions such as blurring, are in the **Process** operation.

Initialize a **Process** instance and define the names of the images that the operation will apply to. In this example, *channel000* contains the nuclear marker and *channel001* contains the reporter. The file names contain these strings. The neural net is only applied to the nuclear marker, so only *channel000* needs to be passed.

Finally, Tte neural net will take some time to run, so it is useful to save the images to a file to skip future runs that may be needed.

In [3]:
pro = ctk.Process(images=['channel000'], output='unet', save=True, force_rerun=False)

Next, add functions to the operation that will be run on the images. *unet_predict* is the function for the convolutional neural net. Different weights can be passed using *weight_path*, but there are also default weights that can be used.

In [4]:
pro.add_function('unet_predict', weight_path='../celltk/config/unet_example_cell_weights.tf')

The output name can be used to pass this output to future operations. Next, **Segment** is used to identify individual cell nuclei.

In [5]:
seg = ctk.Segment(images=['unet'], output='seg', save=True, force_rerun=False)
seg.add_function('constant_thres', thres=0.8)
seg.add_function('clean_labels', min_radius=3, relabel=True)

**Track** is used to connect the segmented objects from frame to frame. Note that the output from the **Segment** operation is saved as a *mask* and needs to be passed to future operations as such.

In [6]:
tra = ctk.Track(images=['channel000'], masks=['seg'], output='nuc', save=True, force_rerun=False)
tra.add_function('kit_sch_ge_tracker')

Finally, **Extract** will get the data from the tracked regions in the images. The output from **Extract** will be an array that will be indexed with the names defining *regions*, *channels*, and *metrics*.  There is a default set of metrics (e.g. area, median_intensity) that will be extracted. Other metrics from skimage.regionprops custom metric functions can be added using **Extract.add_extra_metric**. 

In [7]:
ext = ctk.Extract(images=['channel000', 'channel001'], masks=['nuc'], regions=['nuc'], 
                    channels=['tritc', 'fitc'], time=10, min_trace_length=5, force_rerun=True)

Also available are *derived metrics*. *Derived metrics* refer to calculations that are done with data extracted from the images. For example, a common normalization strategy is to divide the reporter intensity by the marker intensity. The keys index the data in the same way that they will be indexed in the final data structure. *func* can by any *numpy* function.

In [8]:
ext.add_derived_metric('median_ratio',
                       keys=(['nuc', 'fitc', 'median_intensity'],
                             ['nuc', 'tritc', 'median_intensity']),
                       func='divide', inverse=True, propagate=True)

The processing pipeline is now fully defined. The operation objects are passed to the **Pipeline**, which will run the images in the folder through the operations. 

In [9]:
pipe.add_operations([pro, seg, tra, ext])
pipe.run()

2022-06-04 11:23:51,923 - Pipeline - INFO: Added 4 operations.
Current operation list: [<celltk.process.Process object at 0x14b397a00>, <celltk.segment.Segment object at 0x14b3a83a0>, <celltk.track.Track object at 0x14b3a89a0>, <celltk.extract.Extract object at 0x14b3a8550>].
2022-06-04 11:23:51,925 - Pipeline - INFO: Expected inputs: [[('channel000', 'image'), ('unet', 'image')], [('unet', 'image'), ('seg', 'mask')], [('channel000', 'image'), ('seg', 'mask'), ('nuc', 'mask')], [('channel000', 'image'), ('channel001', 'image'), ('nuc', 'mask')]]
2022-06-04 11:23:51,926 - Pipeline - INFO: Exected outputs: [[('unet', 'image')], [('seg', 'mask')], [('nuc', 'mask')], [('data_frame', 'array'), ('data_frame', 'array')]]
2022-06-04 11:23:51,930 - Pipeline - INFO: Looking for nuc of type mask in ../examples/live_cell_example/outputs.
2022-06-04 11:23:51,931 - Pipeline - INFO: Found 0 images.
2022-06-04 11:23:51,932 - Pipeline - INFO: Looking for channel000 of type image in ../examples/live_cel

2022-06-04 11:24:10,327 - Pipeline.Segment - INFO: Adding to save container: seg mask, (6, 720, 853)
2022-06-04 11:24:10,327 - Pipeline.Segment - INFO: clean_labels execution time: 0.7225940227508545
2022-06-04 11:24:10,328 - Pipeline.Segment - INFO: Returning to pipeline: [(('seg', 'mask'), (6, 720, 853), dtype('uint16'))]
2022-06-04 11:24:10,361 - Pipeline - INFO: Saved 6 images in ../examples/live_cell_example/outputs/seg.
2022-06-04 11:24:10,362 - Pipeline.Segment - INFO: Segment execution time: 0.7885780334472656
2022-06-04 11:24:10,363 - Pipeline - INFO: Track at 0x14b3a89a0: ("i:['channel000']", "m:['seg']", 'a:[]') -> kit_sch_ge_tracker() -> mask:nuc
2022-06-04 11:24:10,364 - Pipeline.Track - INFO: Operation Track at 0x14b3a89a0 entered.
2022-06-04 11:24:10,364 - Pipeline.Track - INFO: image:['channel000']
2022-06-04 11:24:10,365 - Pipeline.Track - INFO: mask:['seg']
2022-06-04 11:24:10,366 - Pipeline.Track - INFO: array:[]
2022-06-04 11:24:10,367 - Pipeline.Track - INFO: Outpu

<generator object Operation.run_operation at 0x14c30fac0>